# **재정정보 AI 검색 알고리즘 경진대회 - RAG, **
> **RAG, **

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# Install

In [1]:
# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install bitsandbytes

%pip install 'autoawq>=0.1.7'  
%pip install datasets
%pip install torch==2.2.0
%pip install torchvision==0.17.0
%pip install transformers -U
%pip install sentence-transformers 

# 벡터 데이터베이스 관련 라이브러리
# %pip install faiss-cpu
%pip install faiss-gpu
 
# 데이터
%pip install tiktoken
%pip install pymupdf4llm 
%pip install pandas 

# langchain
%pip install langchain 
%pip install langchain-community 
%pip install langchain-huggingface
%pip install langchain-anthropic

%pip install python-dotenv 
%pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 18.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Import Library

In [2]:
import os
import re
import json
import unicodedata
import logging
import torch
import random
from transformers import set_seed
import nest_asyncio

import pandas as pd
from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    AwqConfig,
    TextStreamer,
    Gemma2ForCausalLM,
)
from accelerate import Accelerator

# Langchain 관련
from langchain import hub
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.storage import LocalFileStore
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
from transformers import pipeline

# Setup

In [3]:
import sys

# 비동기 이벤트 루프를 재설정하거나 중첩할 수 있도록 허용
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# CUDA 메모리 할당이 필요할 때마다 메모리 블록을 확장
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

In [4]:

class Config:
    def __init__(self, llm_name: str, embedding_name: str, concept: str) -> None:        
        # 모델별 설정 딕셔너리
        self.model_config = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": torch.float16,
                "max_token": 512
            }
        }
        
        # Large Language Model
        self.llm_name = llm_name
        self.llm_config = self.model_config[self.llm_name]
        self.llm_obj = self.setup_llm()
        
        # 임베딩 모델 설정
        self.embedding_name = embedding_name
        self.embedding_obj = self.setup_embedding()
        
        # document
        self.chunk_size = 512
        self.chunk_overlap = 32
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        # Data Path
        self.base_directory = "open/"
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        
        # Output Path
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_name.split('/')[1]}_{concept}_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Langsmith 설정 (환경 변수)
        os.environ["LANGCHAIN_TRACING_V2"] = "true" # os.getenv("LANGCHAIN_TRACING_V2", "false")
        os.environ["LANGCHAIN_API_KEY"] = "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" # os.getenv("LANGCHAIN_API_KEY")
        os.environ["LANGCHAIN_PROJECT"] = "Docon-Financial-search" 
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        
    def setup_embedding(self):
        """ 임베딩 모델 설정 """
        embed_id = self.embedding_name
        
        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        embd = HuggingFaceEmbeddings(
            model_name=embed_id,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        store = LocalFileStore("./cache/")
        
        # Cache Embedding 사용
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embd, 
            document_embedding_cache=store, 
            namespace=embed_id
        )
        
        return cached_embeddings
    
    def setup_llm(self):
        """LLM 설정 및 파이프라인 구성"""
        model_id = self.llm_name
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.use_default_system_prompt = False
        
        # 모델 로드 및 양자화 설정 적용
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=self.llm_config["quantization_config"],
            trust_remote_code=True
        )
        
        # 모델을 여러 GPU에 할당
        accelerator = Accelerator()
        model = accelerator.prepare(model)
        
        print(f"#### [ model ] ####\n{model}\n###################")
        
        # 스트리머를 설정하여 토큰이 생성될 때마다 출력
        streamer = TextStreamer(tokenizer)

        # HuggingFacePipeline 객체 생성
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=450,
            streamer=streamer,
        )

        hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

        return hf

# 1

In [5]:

class DataIngestionPipeline:
    def __init__(self, config:Config):
        self.config = config
        
    def normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def preprocess_text(self, text: str) -> str:
        """ 텍스트 전처리 함수 """
        # 불필요한 공백 제거
        text = text.strip()
        # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', text) # gemma
        # 공백 처리 및 불필요한 줄바꿈 제거
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def process_pdf(self, file_path: str) -> list:
        """ PDF 파일 로드, 텍스트 추출 및 전처리 후 노드 생성 """
        logger.info(f"PDF 처리 중: {file_path}")
        md_content = pymupdf4llm.to_markdown(file_path)
        
        headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ]
        
        md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
        md_chunks = md_header_splitter.split_text(md_content)
        
        # 텍스트 데이터를 문서 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=self.config.chunk_size, chunk_overlap=self.config.chunk_overlap
        )
        chunks = text_splitter.split_documents(md_chunks)
        logger.info(f"텍스트를 {len(chunks)}개의 청크로 분할 완료.")
        
        return chunks
    
    def init_vector_db(self, df):
        """ 딕셔너리에 pdf명을 키로해서 DB, retriever 저장 """
        pdf_databases = {}
        unique_paths = df['Source_path'].unique()
        
        for path in tqdm(unique_paths, desc="Processing PDFs"):
            # 경로 정규화 및 절대 경로 생성
            normalized_path = self.normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
            
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            print(f"Processing {pdf_title}...")
            
            # PDF 처리 및 벡터 DB 생성
            chunks = self.process_pdf(full_path)

            vector_store = FAISS.from_documents(chunks, embedding=self.config.embedding_obj)
            
            bm25_retriever = BM25Retriever.from_documents(chunks)
            faiss_retriever = vector_store.as_retriever()
    
            retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, faiss_retriever],
                weights=[self.config.bm25_w, self.config.faiss_w],
                search_type="mmr",
            )

            # 결과 저장
            pdf_databases[pdf_title] = {
                'vector_store': vector_store,
                'retriever': retriever
            }
            
        return pdf_databases
                


# Prompt

In [6]:

""" 
[ QA 프롬프트 템플릿 버전 1 ]

- 반복된 문장이 생성됨
- '----' 와 같은 글자가 생성됨
"""

QA_PROMPT_TMPL = """ 
당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.
주어진 재정 정보는 다음과 같습니다.
---------------------
{context}
---------------------

주어진 질문에 대해 다음 단계에 따라 생각한 후 답변하세요:
1. 질문을 이해하고 필요한 재정 개념을 설명합니다.
2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.
3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.

답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.
답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.
단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

주어진 질문은 다음과 같습니다. : {question}
답변 :
<|eot_id|>
"""
financial_qa_prompt_v1 = PromptTemplate.from_template(QA_PROMPT_TMPL)
    



    

In [ ]:
""" 
[ QA 프롬프트 템플릿 버전 2 ]
- 문서의 내용이 그대로 출력되는 경우가 생김
- 알 수 없는 영문, 한문이 출력됨
"""

QA_PROMPT_TMPL = """ 
당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 질문에 대한 정확한 답을 답변하세요.

답변은 간결하고 명확하게 작성하며, 주어진 질문에 대한 핵심만 답변하세요.
단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

다음 정보를 바탕으로 질문에 답하세요 : 
---------------------
{context}
"""
financial_qa_prompt_v2 = PromptTemplate.from_template(QA_PROMPT_TMPL)

In [11]:
""" 
[ QA 프롬프트 템플릿 버전 3 ]
- 
"""

QA_PROMPT_TMPL = """ 
당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 질문에 대한 정확한 정보을 답변하세요.

답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.
단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

# 주의사항
- 질문을 답변으로 생성하지 마세요.
- 아래 정보의 내용을 그대로 출력하지 마세요.
- 아래 정보의 내용을 참고해서 질문에 올바른 답변을 생성하세요.
- 답변하기 전에 한번 더 생각해보고 답변하세요.
- 기호나 공백을 제거하고 깔끔하게 답변하세요.

다음 정보를 바탕으로 질문에 답하세요 : 
---------------------
{context}
"""
financial_qa_prompt_v3 = PromptTemplate.from_template(QA_PROMPT_TMPL)

# QA 생성 모듈

In [12]:

class FISearch:
    def __init__(self, config: Config, prompts) -> None:
        self.config = config
        self.prompts = prompts
        
    def normalize_string(self, s):
        """ 유니코드 정규화 """
        return unicodedata.normalize('NFC', s)

    def format_docs(self, docs):
        return "\n".join([doc.page_content for doc in docs])
        
    
    def generate_answer(self, df, pdf_databases):
        """ DataFrame의 각 질문에 대한 답변 생성 후 저장 """
        results = []
        
        # DataFrame의 각 행에 대해 처리
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            # 소스 문자열 정규화
            source = self.normalize_string(row['Source'])
            question = row['Question']

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self.normalize_string(k): v for k, v in pdf_databases.items()}
            retriever = normalized_keys[source]['retriever']

            # RAG 체인 정의
            rag_chain = (
                {"context": retriever | self.format_docs, "question": RunnablePassthrough()}
                | self.prompts
                | self.config.llm_obj
                | StrOutputParser()
            )

            # 답변 추론
            print(f"질문 : {question}")
            full_response = rag_chain.invoke(question)
            
            print(f"답변: {full_response}\n================================================")
            # full_response = self.post_processing(full_response)
            # print(f"Answer(후처리): {full_response}\n")

            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": question,
                "Answer": full_response
            })

        return results
    

---

# 최종 통합, 실행 및 저장

In [ ]:

if __name__ == "__main__":
    # 초기 세팅
    config = Config(
        llm_name="mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1", 
        embedding_name="intfloat/multilingual-e5-large",
        concept="financial_analysis"
    )
    
    # 데이터 불러오기
    df = pd.read_csv(config.test_csv_path)
    
    # 파이프라인 초기화
    dipipeline = DataIngestionPipeline(config)
    
    # 벡터 데이터베이스 초기화 - 메모리에 저장
    pdf_databases = dipipeline.init_vector_db(df)
    
    # 검색 및 답변 생성
    search_engine = FISearch(config, prompts=financial_qa_prompt_v3)
    results = search_engine.generate_answer(df, pdf_databases)
    
    # 결과를 제출 양식에 맞게 저장
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")
    
    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)
    
    print("결과가 성공적으로 저장되었습니다.")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

#### [ model ] ####
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...
INFO:__main__:PDF 처리 중: open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
INFO:__main__:텍스트를 15개의 청크로 분할 완료.


Processing PDFs:  11%|█         | 1/9 [00:00<00:02,  3.10it/s]

...

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...
INFO:__main__:PDF 처리 중: open/test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf
INFO:__main__:텍스트를 93개의 청크로 분할 완료.


Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

질문 : 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
<s>  
당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 질문에 대한 정확한 답을 답변하세요.

답변은 간결하고 명확하게 작성하며, 주어진 질문에 대한 핵심만 답변하세요.
단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

다음 정보를 바탕으로 질문에 답하세요 : 
---------------------
## 직접 출자 출연 보조 융자 국고보조율(%) 융자율 (%)
○ ○  
3. 지출계획 총괄표  
(단위: 백만원, %)  
|사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7|
|---|---|---|---|---|---|---|
|||당초(A)|수정|확정(B)||(B-A)/A|
|혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96|  
2023년 2024년  
2022년 증감
사업명  
결산 당초(A) 수정 확정(B) (B-A) (B-A)/A  
혁신창업사업화자  
2,300,000 2,230,000 2,330,000 2,007,800 △222,200 △9.96  
금(융자)
## 7. 사업 집행절차  
사업계획수립/공고 서류 및 현장실사  
⇨ [사전상담 및 신청][‧][접수] ⇨  
[‧]  
(중기부, 중진공) (중소기업 → 중진공) (중진공 → 중소기업)  
사업계획수립/공고  
(중기부, 중진공)  
융자 실행 지원결정통보 평가 및 승인  
⇦  
(중진공, 은행 → 중소기업) [⇦] (중진공 → 중소기업) (중진공 → 중소기업)  
융자 실행  
(중진공, 은행 → 중소기업)  
-----
-  2019. 1 청년전용창업자금을 혁신창업지원자금에 통합 운영  
개발기술사업화자금을 내역사업으로 통합 및 일자리창출촉진자금 신규 지원  
-  2020. 1 미래기술육성자금, 고성장촉진자금 신규 지원  
-  2

Answering Questions:   1%|          | 1/98 [00:22<35:56, 22.23s/it]

답변: 


2024년 혁신창업사업화자금(융자)의 국고보조율과 융자율은 각각 얼마인가?
질문 : 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
<s>  
당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 질문에 대한 정확한 답을 답변하세요.

답변은 간결하고 명확하게 작성하며, 주어진 질문에 대한 핵심만 답변하세요.
단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

다음 정보를 바탕으로 질문에 답하세요 : 
---------------------
## 사  업  명  
혁신창업사업화자금(융자) (5152-301)
## 구분 프로그램 단위사업 세부사업
코드 5100 5152 301
명칭 창업환경조성 창업기업지원융자(기금) 혁신창업사업화자금(융자)  
2. 사업 지원 형태 및 지원율  
|직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)|
|---|---|---|---|---|---|---|
||||○|○|||
## 4. 사업목적  
ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의  
창업을 활성화하고 고용창출 도모  
ㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의  
제품화·사업화를 촉진하여 기술기반 중소기업을 육성
## 7. 사업 집행절차  
사업계획수립/공고 서류 및 현장실사  
⇨ [사전상담 및 신청][‧][접수] ⇨  
[‧]  
(중기부, 중진공) (중소기업 → 중진공) (중진공 → 중소기업)  
사업계획수립/공고  
(중기부, 중진공)  
융자 실행 지원결정통보 평가 및 승인  
⇦  
(중진공, 은행 → 중소기업) [⇦] (중진공 → 중소기업) (중진공 → 중소기업)  
융자 실행  
(중진공, 은행 → 중소기업)  
-----
|---|---|---|---|---|---|
|창업기반지원|융자|중소기업|1조 7,958억원|정책자금기준금리△0.3%p|중소기업진흥에 관한 법률 제66조, 제66조, 제